In [1]:
import numpy as np
import networkx as nx
import pickle
import json
import matplotlib as plt

In [2]:
link_dir = 'http://www-personal.umich.edu/~mejn/netdata/polblogs.zip'
temp_dir = 'data/political/temp/'
data_dir = 'data/political/raw/'
raw_data_filename = 'polblogs.gml'
temp_pickle_graph_filename = 'political_graph.pickle'
ground_truth_filename = 'ground_truth.json'

In [4]:
# largest_SCC, G

In [5]:
G = nx.read_gml("../" + data_dir + raw_data_filename, label='id')
H = G.to_undirected()
largest = max(nx.connected_components(H), key=len)
largest_SCC = H.subgraph(largest)
ground_truths = {
        0: [], # democrat
        1: []  # republican
    }

for i in largest_SCC.nodes:
    ground_truths[largest_SCC.nodes[i]['value']].append(i)

#NOT WORKING FOR SOME REASON, WORKS FOR THE OTHER ONES
nx.write_gpickle(largest_SCC, "../" + temp_dir + temp_pickle_graph_filename)

In [16]:
for i in G.nodes:
#     G.nodes[i]['v'] = []
    print(G.nodes[i])

{'label': '100monkeystyping.com', 'value': 0, 'source': 'Blogarama', 'v': [], 'color': 'blue', '1': 'value'}
{'label': '12thharmonic.com/wordpress', 'value': 0, 'source': 'BlogCatalog', 'v': [], '1': 'value'}
{'label': '40ozblog.blogspot.com', 'value': 0, 'source': 'Blogarama,BlogCatalog', 'v': [], '1': 'value'}
{'label': '4lina.tblog.com', 'value': 0, 'source': 'Blogarama', 'v': [], '1': 'value'}
{'label': '750volts.blogspot.com', 'value': 0, 'source': 'Blogarama', 'v': [], '1': 'value'}
{'label': '95theses.blogspot.com', 'value': 0, 'source': 'Blogarama', 'v': [], '1': 'value'}
{'label': 'abbadabbaduo.blogspot.com', 'value': 0, 'source': 'Blogarama,LeftyDirectory', 'v': [], '1': 'value'}
{'label': 'aboutpolitics.blogspot.com', 'value': 0, 'source': 'Blogarama,eTalkingHead', 'v': [], '1': 'value'}
{'label': 'achingandspinning.blogspot.com', 'value': 0, 'source': 'Blogarama', 'v': [], '1': 'value'}
{'label': 'ackackack.com', 'value': 0, 'source': 'BlogCatalog', 'v': [], '1': 'value'}
{

In [15]:
# nx.set_node_attributes(G, {0: "red", 1: "blue"}, name="color")
nx.set_node_attributes(G, "value", '1')


In [5]:
# number of nodes, number of edges, average degree
# just 0, just 1
sub_graphs = {0:[], 1:[]}
for i in largest_SCC.nodes:
    largest_SCC.subgraph(i)
    
    sub_graphs[largest_SCC.nodes[i]['value']].append(i)

for i in sub_graphs:
    sub_graphs[i] = largest_SCC.subgraph(sub_graphs[i])

sub_graphs

{0: <networkx.classes.multigraph.MultiGraph at 0x7fc190755700>,
 1: <networkx.classes.multigraph.MultiGraph at 0x7fc190755af0>}

In [6]:
def graph_stats(G):
    deg = []
    for i in G.degree:
        deg.append(i[1])
    
    
    stats = {
        'nodes': G.order(),
        'edges': G.number_of_edges(),
        'weights': G.size(),
        'avg_degree': sum(deg) / len(deg)
    }
    
    return stats

In [7]:
# MAIN
print(graph_stats(largest_SCC))

# GROUP 0
print(graph_stats(sub_graphs[0]))

# GROUP 1
print(graph_stats(sub_graphs[1]))


{'nodes': 1222, 'edges': 16782, 'weights': 16782, 'avg_degree': 27.46644844517185}
{'nodes': 586, 'edges': 7322, 'weights': 7322, 'avg_degree': 24.98976109215017}
{'nodes': 636, 'edges': 7880, 'weights': 7880, 'avg_degree': 24.77987421383648}


In [8]:
def spectral_cluster(G):
    A = nx.to_numpy_array(G)
    D = np.diag(A.sum(axis=1))
    # graph laplacian
    L = D-A
    # eigenvalues and eigenvectors
    vals, vecs = np.linalg.eig(L)
    # sort these based on the eigenvalues
    vecs = vecs[:,np.argsort(vals)]
    vals = vals[np.argsort(vals)]
    return [1 if i > 0 else 0 for i in vals]

In [9]:
def prediction_evaluation(G, pred_list):
    # Returns accuracy
    evaluate1 = []
    evaluate2 = []
    index = 0
    for i in list(G.nodes(data = True)):
        evaluate1.append(pred_list[index] == i[1]['value'])
        evaluate2.append(pred_list[index] != i[1]['value'])
        index += 1

    return max(sum(evaluate1)/len(evaluate1), sum(evaluate2)/len(evaluate2))

In [10]:
def spectral_evaluation(G):
    preds = spectral_cluster(G)
    vals = {
        'predictions': spectral_cluster(G),
        'accuracy': prediction_evaluation(G, preds)
    }
    return vals

In [12]:
spectral_evaluation(largest_SCC)["accuracy"]

0.5212765957446809

'data/political/temp/ground_truth.json'

In [ ]:
with open("../" + temp_dir + ground_truth_filename, 'r') as openfile:
        ground_truth_json = json.load(openfile)

In [ ]:
def plot_combined_graph(combined, ground_truth):
    combined = pickle.load(open('data/test/temp/combined.pickle', 'rb'))

    with open('data/test/temp/ground_truth.json', 'r') as openfile:
        ground_truth = json.load(openfile)
  
    pos = nx.spring_layout(combined)
   
    nx.draw_networkx_nodes(combined, pos=pos, nodelist=ground_truth['clique_nodes'], node_color='r', node_size=50)
    nx.draw_networkx_nodes(combined, pos=pos, nodelist=ground_truth['outer_nodes'], node_color='b', node_size=50)
    nx.draw_networkx_edges(combined, pos=pos)
    return plt.show()

In [66]:
def save_graph(graph,file_name):
    #initialze Figure
    plt.figure(num=None, figsize=(50, 50), dpi=100)
    plt.axis('off')
    fig = plt.figure(1)
    pos = nx.spring_layout(graph, seed=22)
    nx.draw_networkx_nodes(graph,pos)
    nx.draw_networkx_edges(graph,pos, alpha=0.5)
    nx.draw_networkx_labels(graph,pos)
    
    plt.savefig(file_name,bbox_inches="tight")
    pylab.close()
    del fig

In [ ]:
def create_original_data():



In [ ]:
def create_prediction_data():


In [ ]:
def create_ground_truth_data():
    

In [ ]:
#miscolored